<a href="https://colab.research.google.com/github/lcipolina/Ray_tutorials/blob/main/WandB_RLLIB_2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ray[rllib] --quiet #ray, version 2.4.0
! ray --version

ray, version 2.4.0


In [ ]:
!pip install wandb --quiet
import wandb
from ray.air.integrations.wandb import WandbLoggerCallback

In [ ]:
!wandb login

wandb: Currently logged in as: lcipolina. Use `wandb login --relogin` to force relogin


In [ ]:
import os
import numpy as np
import ray
#from ray.rllib.env import MultiAgentEnv #RLLIB 2.2
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig

#from ray.rllib.env.multi_agent_env import MultiAgentEnv #new RLLIB 2.3
#from ray.rllib.policy.policy import PolicySpec #For multi-policy mapping

In [ ]:
# New Gym on RLLIB 2.3
!pip install gymnasium --quiet
import gymnasium as gym
#from gym.spaces import MultiDiscrete
#from gym.spaces import Tuple, Box, MultiDiscrete, Discrete

### Configure experiments and train

In [ ]:
from ray.rllib.examples.env.two_step_game import TwoStepGame

In [ ]:
#https://github.com/ray-project/ray/blob/17596b03d189f2e6d642fb5690aefb01faec90d1/rllib/examples/centralized_critic_2.py

#Simplified version

config = PPOConfig()\
        .environment(TwoStepGame)\
        .framework("torch")\
        .rollouts(batch_mode="complete_episodes", num_rollout_workers=0)
      

In [ ]:
callbacks = [WandbLoggerCallback(
	project='WANDB_PROJECT',
        log_config=True)]

In [ ]:
if ray.is_initialized(): ray.shutdown()
ray.init(local_mode=True,include_dashboard=False, ignore_reinit_error=True) 


stop = {
        "training_iteration": 1,
        "timesteps_total": 1
    }

tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(  stop=stop, verbose=1, callbacks=callbacks
                                 ),
       )

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max") #TODO: en realidad, tengo que max por separado
result_df = best_result.metrics_dataframe
print("BEST RESULTS:")
print(result_df[["training_iteration",'episode_reward_mean','episode_len_mean']])

2023-05-02 16:49:31,620	INFO worker.py:1625 -- Started a local Ray instance.


2023-05-02 16:49:34,614	INFO wandb.py:315 -- Already logged into W&B.
:task_name:bundle_reservation_check_func
:actor_name:PPO
2023-05-02 16:49:34,759	WARNING util.py:67 -- Install gputil for GPU system monitoring.


<IPython.core.display.HTML object>
:task_name:bundle_reservation_check_func
:actor_name:PPO
:actor_name:PPO


:actor_name:PPO
2023-05-02 16:50:24,137	WARNING util.py:244 -- The `start_trial` operation took 49.492 s, which may be a performance bottleneck.


TuneError: ignored

### Other things that didn't work

In [ ]:
!from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility #Don't know how this works. Aparently is passed to the config

from: can't read /var/mail/ray.rllib.env.wrappers.multi_agent_env_compatibility


In [ ]:
# Simple MARL custom Environment

'''Simple Multi-agent environment to test things
'''


class TurnEnv(MultiAgentEnv):

    def __init__(self, config=None):
        super().__init__()
        self.num_agents              = 2
        self.t                       = 0
        self.agent_lst               = list(range(self.num_agents))
        self._agent_ids              = set(self.agent_lst)

        # Now RLLIB needs both 'terminated' and 'truncated' to reset the environment
        self.terminateds             = {agent: False for agent in self.agent_lst} #new RLLIB 2.3
        self.terminateds['__all__']  = False
        self.truncateds              = {agent: False for agent in self.agent_lst} #new RLLIB 2.3
        self.truncateds['__all__']   = False
        self.info_dict               = {}

        self.observation_space       = Discrete(self.num_agents)
        self.action_space            = self.observation_space


    def reset(self, *, seed=None, options=None):
        agentID = 0
        self.info_dict = {agentID:
                     {"done": self.terminateds["__all__"]}
                     } #new RLLIB 2.3  
        return {agentID: self.observation_space.sample()},self.info_dict # {agent:obs}, {agent:info}


    def step(self, action_dict):
        
        self.t +=1
        
        # TEST
        #if not action_dict:
        #   print("EMPTY ACTION DICT!!!")
        #   print('self.t =', self.t)
       
        if self.t == 10: 
           self.truncateds             = {agent: True for agent in self.agent_lst}
           self.truncateds['__all__']  = True
        
        
        agentID = 1
        return {agentID:self.observation_space.sample()},\
               {agentID:1},\
               self.terminateds, \
               self.truncateds,\
               self.info_dict
               #obs, rewards, terminateds, truncateds, infos


In [ ]:
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
from ray.tune import register_env
from ray.tune.registry import get_trainable_cls

In [ ]:
#https://github.com/ray-project/ray/blob/master/rllib/examples/env/two_step_game.py

from ray.rllib.env.multi_agent_env import MultiAgentEnv, ENV_STATE

class A_TwoStepGame(MultiAgentEnv):
    action_space = Discrete(2)

    def __init__(self, env_config):
        super().__init__()
        self.action_space = Discrete(2)
        self.state = None
        self.agent_1 = 0
        self.agent_2 = 1
        self._skip_env_checking = True
        # MADDPG emits action logits instead of actual discrete actions
        self.actions_are_logits = env_config.get("actions_are_logits", False)
        self.one_hot_state_encoding = env_config.get("one_hot_state_encoding", False)
        self.with_state = env_config.get("separate_state_space", False)
        self._agent_ids = {0, 1}
        if not self.one_hot_state_encoding:
            self.observation_space = Discrete(6)
            self.with_state = False
        else:
            # Each agent gets the full state (one-hot encoding of which of the
            # three states are active) as input with the receiving agent's
            # ID (1 or 2) concatenated onto the end.
            if self.with_state:
                self.observation_space = Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )
            else:
                self.observation_space = MultiDiscrete([2, 2, 2, 3])

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.state = np.array([1, 0, 0])
        return self._obs(), {}

    def step(self, action_dict):
        if self.actions_are_logits:
            action_dict = {
                k: np.random.choice([0, 1], p=v) for k, v in action_dict.items()
            }

        state_index = np.flatnonzero(self.state)
        if state_index == 0:
            action = action_dict[self.agent_1]
            assert action in [0, 1], action
            if action == 0:
                self.state = np.array([0, 1, 0])
            else:
                self.state = np.array([0, 0, 1])
            global_rew = 0
            terminated = False
        elif state_index == 1:
            global_rew = 7
            terminated = True
        else:
            if action_dict[self.agent_1] == 0 and action_dict[self.agent_2] == 0:
                global_rew = 0
            elif action_dict[self.agent_1] == 1 and action_dict[self.agent_2] == 1:
                global_rew = 8
            else:
                global_rew = 1
            terminated = True

        rewards = {self.agent_1: global_rew / 2.0, self.agent_2: global_rew / 2.0}
        obs = self._obs()
        terminateds = {"__all__": terminated}
        truncateds = {"__all__": False}
        infos = {
            self.agent_1: {"done": terminateds["__all__"]},
            self.agent_2: {"done": terminateds["__all__"]},
        }
        return obs, rewards, terminateds, truncateds, infos

    def _obs(self):
        if self.with_state:
            return {
                self.agent_1: {"obs": self.agent_1_obs(), ENV_STATE: self.state},
                self.agent_2: {"obs": self.agent_2_obs(), ENV_STATE: self.state},
            }
        else:
            return {self.agent_1: self.agent_1_obs(), self.agent_2: self.agent_2_obs()}

    def agent_1_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [1]])
        else:
            return np.flatnonzero(self.state)[0]

    def agent_2_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [2]])
        else:
            return np.flatnonzero(self.state)[0] + 3
